# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [8]:
# Load pickled data
import tensorflow as tf
from hashlib import md5
import pandas as pd 

import numpy as np
import pickle
import os
import time
from importlib import reload
from pprint import pprint


import helpers as h 
import architectures as arch

if os.name == "nt" : # running on windows
    DATA_DIR = "C:/_DATA/autonomous-driving-nd/traffic_signs_classifier/"
    MY_DEV = "/cpu:0"
else : 
    DATA_DIR = "../data/"
    MY_DEV = "/gpu:0"
# These files were downloaded from the data directory in the workshop enviornment
training_file   = DATA_DIR + "train.p"
validation_file = DATA_DIR + "valid.p"
testing_file    = DATA_DIR + "test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train0, y_train = train['features'], train['labels']
X_valid0, y_valid = valid['features'], valid['labels']
X_test0, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [2]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = X_train0.shape[0]

# TODO: Number of validation examples
n_validation = X_valid0.shape[0]

# TODO: Number of testing examples.
n_test = X_test0.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = X_train0.shape[1:3]

# TODO: How many unique classes/labels there are in the dataset.
uniq_classes = np.unique( np.hstack( [y_train, y_test, y_valid] ) ) 
n_classes = len( uniq_classes ) 

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes,  " min_class =", np.min(uniq_classes), " max_class =", np.max(uniq_classes))

Number of training examples = 34799
Number of testing examples = 12630
Image data shape = (32, 32)
Number of classes = 43  min_class = 0  max_class = 42


### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
import pandas as pd 

# Visualizations will be shown in the notebook.
%matplotlib inline

In [ ]:
n_examples = 7
example_indices = np.zeros( (n_classes, n_examples), dtype=np.int32)

for cls in uniq_classes : 
    all_ = np.where( y_train  == cls )[0]
    example_indices[cls, :] = all_[ np.random.choice(len(all_), n_examples) ]


def plot_examples_classes( example_indices, X_train, min_cls, max_cls ) :
    n_examples = example_indices.shape[1]

    fig, axs = plt.subplots( max_cls - min_cls, n_examples, figsize=(10,10) )
    fig.suptitle( f'Classes {min_cls} to {max_cls -1}')
    for cls in range(min_cls, max_cls): 
        for ex_i in range(n_examples) :
            ax = axs[cls - min_cls, ex_i]
            img_idx = example_indices[cls, ex_i]
            ax.imshow( X_train[ img_idx,...] )
            ax.tick_params( axis='x', which='both', bottom=False,  top=False,  labelbottom=False) 
            ax.tick_params( axis='y', which='both', left=False,    labelleft=False) 


In [ ]:
plot_examples_classes( example_indices, X_train, 0, 10 )

In [ ]:
plot_examples_classes( example_indices, X_train, 10, 20 )

In [ ]:
plot_examples_classes( example_indices, X_train, 20, 32 )

In [ ]:
plot_examples_classes( example_indices, X_train, 32, 43 )

In [ ]:
import pandas as pd
y_tr_df = pd.DataFrame( {'cls' : y_train, 'train_cnt' : 1}) 
y_vl_df = pd.DataFrame( {'cls' : y_valid, 'valid_cnt' : 1}) 
y_ts_df = pd.DataFrame( {'cls' : y_test, 'test_cnt' : 1}) 

In [ ]:
#fig, axs = plt.subplots( 1,3, figsize=(16,5) ) 
tr_cnts, vl_cnts, ts_cnts  = [ df.groupby('cls').sum() 
                              for df in [y_tr_df, y_vl_df, y_ts_df] ]

joined = tr_cnts.join( vl_cnts ).join( ts_cnts )

for col in joined.columns : 
    joined[ col.replace('cnt', 'pct') ] = joined[col] / joined[col].sum() * 100.0 


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,10))
ax.set_title('Pct of signs ')
joined[['train_pct', 'valid_pct', 'test_pct']].plot( kind='barh', ax=ax)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
import cv2

In [3]:
X_train0[0].dtype

dtype('uint8')

In [ ]:
img  = X_valid0[1]  # 245

def preproc( img ) :
    X_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV  )
    X_hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS  )

    h, s, v  = X_hsv[:,:,0], X_hsv[:,:,1], X_hsv[:,:,2]
    h2, l, s2  = X_hls[:,:,0], X_hls[:,:,1], X_hls[:,:,2]

    edges_h = cv2.Canny(h,100,200)
    edges_s = cv2.Canny(s,100,180)
    
    return [ img, h,s,v, l, edges_s, edges_s.sum(axis=0), edges_s.sum(axis=1) ]

imgs = [X_valid0[i] for i in np.random.choice( X_valid0.shape[0], 10 ) ]
results = [ preproc(img) for img in imgs ]

n_images = 10

fig, axs = plt.subplots( n_images, 8, figsize=(16,16) )
for i, layers in enumerate( results ) :
    axs[i, 0].imshow( layers[0] )
    axs[i, 1].imshow( layers[1], cmap = 'gray')
    axs[i, 2].imshow( layers[2], cmap = 'gray')
    axs[i, 3].imshow( layers[3], cmap = 'gray')
    #axs[i,4].imshow( l, cmap = 'gray')
    axs[i, 4].imshow( layers[4], cmap = 'gray')
    #axs[i, 5].imshow( s - (v-l), cmap = 'gray')
    #axs[i, 5].imshow( layers[5], cmap = 'gray')
    axs[i, 5].imshow( layers[5], cmap = 'gray')
    axs[i, 7].plot( layers[6] )
    axs[i, 6].plot( layers[7] )


In [ ]:
plt.hist( s.flatten() ) 

In [3]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

import helpers as h 
import architectures as arch

reload(h)

#X_train, X_valid, X_test = [ h.preproc_hs( X ).astype( np.float32 ) for X in [X_train0, X_valid0, X_test0] ]
X_train, X_valid, X_test = [ h.normalize_mean_std( X ).astype( np.float32 ) for X in [X_train0, X_valid0, X_test0] ]
X_train.shape, X_valid.shape, X_test.shape, X_train.dtype

((34799, 32, 32, 3), (4410, 32, 32, 3), (12630, 32, 32, 3), dtype('float32'))

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

In [7]:
arch_3_3_c = [ None, # index=0 won't be used
    { 'type'  : 'conv2d', 'W_pars' : ( 3, 3, 64),  'strides' : ( 1, 1, 1, 1),
      'name'  : 'conv1'},

    {  'type'    : 'max_pool', 'ksize'   : (1, 2, 2, 1), 'strides' : (1, 2, 2, 1),
       'padding' : 'SAME', 'name'    : 'max_p1'  },

    { 'type'  : 'conv2d', 'W_pars' : ( 3, 3, 32),  'strides' : ( 1, 1, 1, 1),
      'name'  : 'conv2'},

    { 'type'    : 'max_pool', 'ksize'   : (1, 2, 2, 1), 'strides' : (1, 2, 2, 1),
      'padding' : 'SAME', 'name'    : 'max_p2'  },

    { 'type'  : 'conv2d', 'W_pars' : ( 5, 5, 16),  'strides' : ( 1, 1, 1, 1),
      'name'  : 'conv3'},

    # layer 4 : max_pool
    {  'type'    : 'max_pool', 'ksize' : (1, 2, 2, 1), 'strides' : (1, 2, 2, 1),
       'padding' : 'SAME',   'name'  : 'max_p3'  },
     # layer 5 : flatten
    {  'type'    : 'flatten', 'name'    : 'flat1'},
    #layer 6 : fully_connected
    {  'type'    : 'fully_connected', 'out_dim' : 120, 'nonlinear' : tf.nn.relu, 'name'    : 'fc1'},
    #layer 7 : fully_connected
    {  'type'    : 'fully_connected', 'out_dim' : 84,  'nonlinear' : tf.nn.relu, 'name' : 'fc2'},
    #layer 8 : fully_connected  - no relu afterwards
    {  'type'    : 'fully_connected', 'out_dim' : 43,  'nonlinear' : None,   'name' : 'fc3' }
    ]


In [5]:
del X_train0, X_valid0, X_test0

In [5]:
# tf Graph input
from sklearn.utils import shuffle
reload( h )
reload( arch )

data = { "X_train" : X_train, "y_train" : y_train,
         "X_valid" : X_valid, "y_valid" : y_valid,
         "X_test" : X_test, "y_test" : y_test     }

hyp_pars = {
    "netw_arch_name" : "arch_3_3",
    #netw_arch = arch.arch_3_3, bs 256, lr 1e-4
    "learning_rate" : 0.0002,
    "keep_prob" : 0.5,    
    "batch_size" : 5000, #256     
}

if os.name != 'nt' :
    log_pars = { "print_loss_every" : 300,
                 "run_valid_every"  : 300 }
else : 
    log_pars = { "print_loss_every" : 3,
                 "run_valid_every"  : 10 }
    


In [11]:
def md5_digest_from_pars( hyp_pars, digest_len = 8) : 
    hyp_pars_str = str( sorted( list( hyp_pars.items() ) ) ) 
    md5_dig = md5( hyp_pars_str.encode("utf8") ).hexdigest()[:digest_len]
    out_path = "experiment_results/exp_" + md5_dig + ".pkl"
    return md5_dig, out_path 

md5_dig, out_path = md5_digest_from_pars( hyp_pars )
md5_dig, out_path

('09373329', 'experiment_results/exp_09373329.pkl')

True

In [ ]:
reload( h )
lattice_specs = [
    ( "netw_arch_name", ["arch_3_3", "arch_3_3_2fc", "arch_3_3_b"] ),
    ( "learning_rate" , [ 0.0005, 0.0010 ] ),                         
    ( "batch_size" , [ 256, 512] ),
    ( "keep_prob"  , [ 0.9, 0.8, 0.7] ),    
]

def to_pickle( obj, fname ) : 
    with open( fname, "wb") as f_out :
        print( "Writing to " + fname )
        pickle.dump( obj, f_out )

hyp_par_dicts = h.make_hyp_par_dicts( lattice_specs )

from pprint import pprint

for exp_i, hyp_pars in enumerate( hyp_par_dicts ) :
    print( f"\n Experiment {exp_i + 1}/{len(hyp_par_dicts)} :\n")
    
    pprint( hyp_pars )    
    result = hyp_pars.copy()
    
    md5_dig, out_path = md5_digest_from_pars( hyp_pars )
    
    if os.path.exists( out_path ) :
        print( out_path + " already there.")
        continue 
    
    result["md5_dig"] = md5_dig
    per_epoch = h.run_training( data, hyp_pars, log_pars, n_epochs = 200) 
    #print("\n", per_epoch, "\n")
    
    result["best_valid_accy"] = per_epoch["valid_accy"].max()    
    result["best_valid_epoch"] = per_epoch['valid_accy'].idxmax()
    
    pprint( hyp_pars )
    result["per_epoch"] = per_epoch    
    to_pickle( result,  out_path )


 Experiment 1/36 :

{'batch_size': 256,
 'keep_prob': 0.9,
 'learning_rate': 0.0005,
 'netw_arch_name': 'arch_3_3'}
experiment_results/exp_3eb42e7b.pklalready there.

 Experiment 2/36 :

{'batch_size': 256,
 'keep_prob': 0.8,
 'learning_rate': 0.0005,
 'netw_arch_name': 'arch_3_3'}
experiment_results/exp_51f797a7.pklalready there.

 Experiment 3/36 :

{'batch_size': 256,
 'keep_prob': 0.7,
 'learning_rate': 0.0005,
 'netw_arch_name': 'arch_3_3'}
experiment_results/exp_eb00d42f.pklalready there.

 Experiment 4/36 :

{'batch_size': 512,
 'keep_prob': 0.9,
 'learning_rate': 0.0005,
 'netw_arch_name': 'arch_3_3'}
experiment_results/exp_ba8cc667.pklalready there.

 Experiment 5/36 :

{'batch_size': 512,
 'keep_prob': 0.8,
 'learning_rate': 0.0005,
 'netw_arch_name': 'arch_3_3'}
experiment_results/exp_a5ab239c.pklalready there.

 Experiment 6/36 :

{'batch_size': 512,
 'keep_prob': 0.7,
 'learning_rate': 0.0005,
 'netw_arch_name': 'arch_3_3'}
experiment_results/exp_2b5d0361.pklalready there

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")